In [39]:
from fuzzywuzzy import fuzz
import pandas as pd
from numpy import triu, ones
import numpy as np

In [40]:
with open("product_list2.csv", "r") as fp:
    data_tgt = fp.readlines()

df = pd.read_csv("scraped list.csv")
    
list_tgt = [x.replace('"',"") for x in list(data_tgt)]
list_src = df['product-name'].values.tolist()[:50]

In [41]:
def clean(s):
    d = {}
    s = s.lower()
    sl = s.split(" ")
    d.update({'brand': sl[0], 'type': sl[1]})
    second = sl[2:]
    for part in second:
        if "mb/s" in part:
            d.update({'speed':part})
        if "gb" in part:
            d.update({'capacity':part.replace('\n', "")})
            
    return d

In [42]:
prodict = {}
for l in list_tgt:
    prodict.update({l: clean(l)})

In [43]:
df = pd.DataFrame(prodict).T
df.head()

,brand,capacity,speed,type
KINGSTON CF CARD 4GB\n,kingston,4gb,NaN,cf
KINGSTON CF CARD 8GB\n,kingston,8gb,NaN,cf
KINGSTON SDC CLASS 10 80MB/S 128GB\n,kingston,128gb,80mb/s,sdc
KINGSTON SDC CLASS 10 80MB/S 16GB\n,kingston,16gb,80mb/s,sdc
KINGSTON SDC CLASS 10 80MB/S 256MB\n,kingston,NaN,80mb/s,sdc


In [44]:
df.isnull().sum()

brand        0
capacity     1
speed       19
type         0
dtype: int64

In [45]:
type(df.iloc[0,0])

str

In [52]:
df_temp = df.applymap(lambda x: 1 if type(x) == str and x in list_src[0].lower() else 0)
df_temp.sum(axis=1).sort_values().index.tolist()[0]

'KINGSTON CF CARD 4GB\n'

In [55]:
matchdict = {}
for s in list_src:
    df_temp = df.applymap(lambda x: 1 if type(x) == str and x in s.lower() else 0)
    key = df_temp.sum(axis=1).sort_values(ascending=False).index.tolist()[0]
    matchdict.setdefault(key, []).append(s)

In [56]:
matchdict

{'KINGSTON CF CARD 4GB\n': ['Transcend Rdf9 Usb3.1/3.0 Sd Uhs Ii And Cf Card Reader',
  'Kingston Microsdxc 64Gb Uhs I U3 90R/80W Memory Card',
  'Oanda Professional 4Gb Compact Flash Memory Card(Whiteblue)',
  'High Speed Usb 3.0 All In 1 Sd Tf Cf Xd M2 Ms Flash Memory Cardreader Adapter'],
 'KINGSTON CF CARD 8GB\n': ['Transcend Cfx650 128Gb Cfast 2.0 Flash Memory Card'],
 'KINGSTON SDHC CLASS 10 80MB/S SERIES 64GB\n': ['Samsung Evo Plus Micro Sdhc Ush 1 U3 64Gb Class10 Fhd 100M/S   Red  Intl'],
 'KINGSTON SDHC CLASS 4 32GB\n': ['Kingston Microsdhc 32Gb Uhs I U3 90R/80W Memory Card'],
 'SAMSUNG SDC EVO CLASS 10 100MB/S 128GB\n': ['Samsung Evo Plus Micro Sdxc Class 10 Uhs 1 64G Memory Card',
  'Samsung 128Gb Evo Plus Microsd Class 10 W Adapter (Mb Mc128Daapc)',
  'Samsung Evo 128Gb Microsdxc Uhs-I U3 With Sd Adapter Mb Mp128Ga  Intl'],
 'SAMSUNG SDC EVO CLASS 10 100MB/S 64GB\n': ['Samsung 2017 Evo Plus Micro Sd Tf Memory Card 64Gb',
  'Samsung Evo 64Gb Class 10 U3 100Mb/S 4K Ultra Hd M